In [1]:
import pandas as pd
from utils import select_min_pay, read_input_file, export_file
import glob
import numpy as np
import pandas as pd
import glob
import logging
import win32com.client as win32
import os
import jaydebeapi
from const import QUERY, CONNECT

In [2]:
Acc = glob.glob("./input/Acc_*.csv")
Acc_date = Acc[0].split('_')[1]

In [4]:
def select_min_pay():
    conn = jaydebeapi.connect(
        CONNECT.DB,
        CONNECT.PORT,
        CONNECT.USER,
        CONNECT.CNN,
        )
    # cur = conn.cursor()  
    

    min_pay = pd.read_sql(QUERY.MIN_PAY, conn) 
    print('QUERY.MIN_PAY',QUERY.MIN_PAY)
    conn.close()
    return min_pay

In [5]:
select_min_pay()


C:\Users\Sireetron\AppData\Local\Temp\ipykernel_33212\2699873309.py:11: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  min_pay = pd.read_sql(QUERY.MIN_PAY, conn)


QUERY.MIN_PAY 
     WITH lastdate AS (
SELECT LOAN_NO, MAX(SETL_DT) AS CURRENT_DUE FROM INSTC.CFNC_INDM_L GROUP BY LOAN_NO
)
SELECT 
        a.LOAN_NO, 
        b.CURRENT_DUE,
        NVL(a.INS_THMM_DMND_PCPL_AMT,0) 
        + NVL(a.INS_THMM_DMND_INT_AMT,0) 
        + NVL(a.INS_THMM_DMND_CRED_USAGE_AMT,0)
        + NVL(a.INS_THMM_CLCF_AMT,0)
        + NVL(a.INS_THMM_VAT_AMT,0)  AS LAST_DUE_AMT 
    FROM INSTC.CFNC_INDM_L  a
    INNER JOIN lastdate b 
        ON a.LOAN_NO = b.LOAN_NO 
        AND a.SETL_DT = b.CURRENT_DUE 
    


,LOAN_NO,CURRENT_DUE,LAST_DUE_AMT
0,9000220022000043,20210917,1641.00
1,9000220022000051,20201117,0.00
2,9000220022000078,20210317,0.00
3,9000220022000086,20210417,0.00
4,9000220022000094,20210317,0.00
...,...,...,...
1879399,7231270101836253,20250301,360.28
1879400,7231270101836585,20250301,1077.94
1879401,7231270101846264,20250301,756.06
1879402,7231270101830597,20250301,1196.33


In [50]:
MIN_PAY

,LOAN_NO,CURRENT_DUE,LAST_DUE_AMT
0,9000220022000043,20210917,1641.00
1,9000220022000051,20201117,0.00
2,9000220022000078,20210317,0.00
3,9000220022000086,20210417,0.00
4,9000220022000094,20210317,0.00
...,...,...,...
1879399,7231270101836253,20250301,360.28
1879400,7231270101836585,20250301,1077.94
1879401,7231270101846264,20250301,756.06
1879402,7231270101830597,20250301,1196.33


In [51]:
logging.basicConfig(
    level=logging.INFO, format="%(asctime)s - %(levelname)s - %(message)s"
)
# Create a logger
logger = logging.getLogger(__name__)


In [52]:
files = glob.glob("./Input/*.*")
files

['./Input\\Acc_20240826_0830.csv',
 './Input\\Code min pay.txt',
 './Input\\GPD SPD.xlsx',
 './Input\\Input min pay.xlsx',
 "./Input\\Map Owner Aug'24 (26082024).xlsx",
 './Input\\New Text Document.txt']

In [53]:
AR, AR_TIME, GPD_SPD, OWNER = read_input_file()

2025-03-07 15:41:39,197 - INFO - Start Processing...
2025-03-07 15:41:39,199 - INFO - Reading input files...
2025-03-07 15:41:39,200 - INFO - Reading AR file...
2025-03-07 15:41:42,922 - INFO - Reading GPD SPD file...
2025-03-07 15:41:43,539 - INFO - Reading Map Owner file...


In [54]:
OWNER

,LOAN_NO,OA,Due
0,5001759091041047,KWC,1
1,5000159121051930,MGC,1
2,5001060081060501,PCN,1
3,5000161031071705,PCN,1
4,8002059091074221,PCN,1
...,...,...,...
134691,7230870101096234,SMS,1
134692,7230770100899446,SMS,1
134693,7230770100850851,SMS,1
134694,7230870100971173,SMS,1


In [55]:
AR['LOAN_NO']  = AR['LOAN_NO'].astype('O')

In [56]:
AR = AR.merge(OWNER, on='LOAN_NO', how='left')
AR = AR.merge(GPD_SPD, on='LOAN_NO', how='left')
AR = AR.merge(MIN_PAY, on='LOAN_NO', how='left')


In [57]:
AR

,BaseDate,LOAN_NO,Product Code,Overdue Start Date,OverdueDays_Morning,OverdueCnt_Morning,OverdueDays,OverdueCnt,Loan Amt,Loan Balance,...,OverdueNCBTaxAmt,OverdueETCFeeAmt,LawCost,LastPaymentDt,LastPaymentAmt,OA,Due,GPD/SPD,CURRENT_DUE,LAST_DUE_AMT
0,20240826,FB1910045_220309,NaN,2022-09-01,725,12:OVER,725,12:OVER,11547990.00,6822230.13,...,0,0.0,0.0,2024-05-07,60000.0,NaN,NaN,NaN,20250101,0.0
1,20240826,9010221122000949,A501:TL,2024-03-01,178,6:OD5,178,6:OD5,74164.44,49861.75,...,0,0.0,0.0,2024-06-27,2300.0,GMV,1,NaN,20250301,3067.0
2,20240826,9010221122000931,A501:TL,,0,99:NORMAL,0,99:NORMAL,41659.76,21883.20,...,0,0.0,0.0,2024-08-01,1667.0,NaN,NaN,NaN,20250301,1667.0
3,20240826,9010221122000922,A501:TL,2023-08-31,0,98:Write-Off,0,98:Write-Off,34238.99,23851.00,...,0,0.0,0.0,2023-01-31,1300.0,NaN,NaN,NaN,20230901,1407.0
4,20240826,9010221122000914,A501:TL,2024-02-29,0,98:Write-Off,0,98:Write-Off,16118.33,13559.55,...,0,0.0,0.0,2023-12-27,550.0,NaN,NaN,NaN,20240301,505.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1239321,20240826,1000158090014831,A401:ML,2017-02-01,0,99:NORMAL,0,99:NORMAL,25500.00,100.00,...,0,0.0,0.0,2017-01-31,1580.0,NaN,NaN,NaN,20250301,0.0
1239322,20240826,1000158090014814,A401:ML,2017-08-01,0,99:NORMAL,0,99:NORMAL,24600.00,30.00,...,0,0.0,0.0,2017-07-31,1260.0,NaN,NaN,NaN,20250301,0.0
1239323,20240826,1000158090008580,A401:ML,2017-08-01,0,99:NORMAL,0,99:NORMAL,25900.00,100.00,...,0,0.0,0.0,2017-07-16,4000.0,NaN,NaN,NaN,20250301,0.0
1239324,20240826,1000158090005394,A401:ML,2017-08-01,0,99:NORMAL,0,99:NORMAL,25900.00,100.00,...,0,0.0,0.0,2017-08-01,2600.0,NaN,NaN,NaN,20250301,0.0


In [58]:
AR.loc[AR['Product Code'].str.contains('SF', na=False)]

,BaseDate,LOAN_NO,Product Code,Overdue Start Date,OverdueDays_Morning,OverdueCnt_Morning,OverdueDays,OverdueCnt,Loan Amt,Loan Balance,...,OverdueNCBTaxAmt,OverdueETCFeeAmt,LawCost,LastPaymentDt,LastPaymentAmt,OA,Due,GPD/SPD,CURRENT_DUE,LAST_DUE_AMT
10301,20240826,7240870103631165,A701:SF,,0,99:NORMAL,0,99:NORMAL,6073.08,6073.08,...,0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,20250301,472.98
10302,20240826,7240870103631152,A701:SF,,0,99:NORMAL,0,99:NORMAL,3974.84,3974.84,...,0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,20250301,309.56
10303,20240826,7240870103631151,A701:SF,,0,99:NORMAL,0,99:NORMAL,8944.63,8944.63,...,0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,20250301,796.62
10304,20240826,7240870103631150,A701:SF,,0,99:NORMAL,0,99:NORMAL,7067.28,7067.28,...,0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,20250301,550.41
10305,20240826,7240870103631130,A701:SF,,0,99:NORMAL,0,99:NORMAL,10177.14,10177.14,...,0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,20250301,528.40
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1127113,20240826,7221070100000545,A701:SF,2024-04-30,0,98:Write-Off,0,98:Write-Off,6521.17,519.42,...,0,0.0,0.0,2023-10-02,552.12,NaN,NaN,NaN,20240501,0.00
1127114,20240826,7221070100000475,A701:SF,,0,99:NORMAL,0,99:NORMAL,36992.96,3127.61,...,0,0.0,0.0,2024-08-01,1563.88,NaN,NaN,NaN,20241001,0.00
1127115,20240826,7221070100000461,A701:SF,,0,99:NORMAL,0,99:NORMAL,32507.35,2748.59,...,0,0.0,0.0,2024-07-31,1374.25,NaN,NaN,NaN,20241101,0.00
1127116,20240826,7221070100000449,A701:SF,,0,99:NORMAL,0,99:NORMAL,11473.18,970.08,...,0,0.0,0.0,2024-08-01,485.03,NaN,NaN,NaN,20241101,0.00


In [59]:
not_map = ['NORMAL', 'Normal', 'normal', 'XD', 'Write-Off']
pattern = '|'.join(not_map)
AR['LAST_DUE_AMT'] = np.where(AR['Product Code'].str.contains('SF', na=False), None, AR['LAST_DUE_AMT'])


In [60]:
AR['LAST_DUE_AMT'] = np.where(AR['OverdueCnt'].str.contains(pattern, na=False), None, AR['LAST_DUE_AMT'])

In [37]:
def set_excel_file_password(excel_file_path, password):
    # Create an Excel application object
    excel_app = win32.Dispatch("Excel.Application")

    # Convert to absolute path
    excel_file_path = os.path.abspath(excel_file_path)

    # Open the Excel file
    wb = excel_app.Workbooks.Open(excel_file_path)

    # Set a password for the entire workbook
    wb.Password = password

    # Save and close the workbook
    wb.Save()
    wb.Close()

In [38]:
AR_SF = AR[AR['Product Code'].str.contains('SF', na=False)].reset_index(drop=True)
AR_ALL = AR[~AR['Product Code'].str.contains('SF', na=False)]
AR_SF1 = AR_SF[:1000001]
AR_SF2 = AR_SF[1000001:]
print('========finishsprit========')
AR_SF_ASSIGN = AR_SF[~AR_SF['OA'].isna()]
logger.info('Exportting AR and AR_SF..')

with pd.ExcelWriter(f'./Output/AR_TOTAL/AR {Acc_date} Time {AR_TIME}.xlsx', engine='xlsxwriter') as writer:
    AR_ALL.to_excel(writer, index=False, sheet_name='AR_ALL')
    print('========finishALL========')
    AR_SF1.to_excel(writer, index=False, sheet_name='AR_SF1')
    print('========finishSF1========') 
    AR_SF2.to_excel(writer, index=False, sheet_name='AR_SF1')
    print('========finishSF2========') 
    AR_SF_ASSIGN.to_excel(writer, index=False, sheet_name='AR_SF_ASSIGN')

2025-03-05 08:52:31,159 - INFO - Exportting AR and AR_SF..


========finishsprit========
========finishALL========
========finishSF1========
========finishSF2========


In [ ]:
AR_SF = AR[AR['Product Code'].str.contains('SF', na=False)].reset_index(drop=True)
AR_ALL = AR[~AR['Product Code'].str.contains('SF', na=False)]
split_point = len(AR_SF) //2  # Splitting roughly in half
AR_SF1 = AR_SF.iloc[:split_point]
AR_SF2 = AR_SF.iloc[split_point:]

In [57]:
max_rows = 500000

# Calculate the number of splits needed
num_splits = (len(AR_SF) // max_rows) + (1 if len(AR_SF) % max_rows > 0 else 0)

# Create a list to store the split DataFrames
split_dataframes = []

# Split the DataFrame into chunks
for i in range(num_splits):
    start_idx = i * max_rows
    end_idx = min((i + 1) * max_rows, len(AR_SF))  # Ensure the last part handles remaining rows
    split_dataframes.append(AR_SF.iloc[start_idx:end_idx])


In [60]:
for idx, df in enumerate(split_dataframes, start=1):
    print(f"DataFrame SF{idx}:")
    print(df)
    print("\n" + "-"*50 + "\n")

DataFrame SF1:
        BaseDate           LOAN_NO Product Code Overdue Start Date  \
0       20240826  7240870103631165      A701:SF                      
1       20240826  7240870103631152      A701:SF                      
2       20240826  7240870103631151      A701:SF                      
3       20240826  7240870103631150      A701:SF                      
4       20240826  7240870103631130      A701:SF                      
...          ...               ...          ...                ...   
499995  20240826  7240370102291249      A701:SF                      
499996  20240826  7240370102291247      A701:SF                      
499997  20240826  7240370102291245      A701:SF                      
499998  20240826  7240370102291242      A701:SF                      
499999  20240826  7240370102291238      A701:SF                      

        OverdueDays_Morning OverdueCnt_Morning  OverdueDays OverdueCnt  \
0                         0          99:NORMAL            0  99:NORMAL

In [ ]:
len(AR_ALL)

In [61]:
import time
AR_SF = AR[AR['Product Code'].str.contains('SF', na=False)].reset_index(drop=True)
AR_ALL = AR[~AR['Product Code'].str.contains('SF', na=False)]
split_point = len(AR_SF)//2   # Splitting roughly in half
AR_SF1 = AR_SF.iloc[:split_point]
AR_SF2 = AR_SF.iloc[split_point:]

print('========finishsprit========')
AR_SF_ASSIGN = AR_SF[~AR_SF['OA'].isna()]
logger.info('Exportting AR and AR_SF..')
start_time = time.time()
print('========startsave========')

with pd.ExcelWriter(f'./Output/AR_TOTAL/AR {Acc_date} Time {AR_TIME}.xlsx', engine='xlsxwriter') as writer:
    step_start = time.time()
    AR_ALL.to_excel(writer, index=False, sheet_name='AR_ALL')
    print(f'Finished AR_ALL in {time.time() - step_start:.2f} seconds')
    
    step_start = time.time()
    AR_SF1.to_excel(writer, index=False, sheet_name='AR_SF1')
    print(f'Finished AR_SF1 in {time.time() - step_start:.2f} seconds')

    step_start = time.time()
    AR_SF2.to_excel(writer, index=False, sheet_name='AR_SF2')  # Fixed duplicate sheet name
    print(f'Finished AR_SF2 in {time.time() - step_start:.2f} seconds')

    step_start = time.time()
    AR_SF_ASSIGN.to_excel(writer, index=False, sheet_name='AR_SF_ASSIGN')
    print(f'Finished AR_SF_ASSIGN in {time.time() - step_start:.2f} seconds')

    end_time = time.time()
print(f'Total execution time: {end_time - start_time:.2f} seconds')


2025-03-07 15:43:46,070 - INFO - Exportting AR and AR_SF..


========finishsprit========
========startsave========
Finished AR_ALL in 25.20 seconds
Finished AR_SF1 in 77.39 seconds


KeyboardInterrupt: 

In [54]:
AR_OA = AR[~AR['OA'].isna()]
OA = AR_OA['OA'].unique()
password = input('กรุณาระบุ password: ')

for owner in OA:
        
    print('========owner========',owner)
    OA_data = AR_OA[AR_OA['OA'] == owner]
    OA_SF = OA_data[OA_data['Product Code'].str.contains('SF', na=False)]
    OA_data = OA_data[~OA_data['Product Code'].str.contains('SF', na=False)]

    
    if OA_SF.shape[0] == 0:
        with pd.ExcelWriter(f'./Output/AR_OA/AR {Acc_date} Time {AR_TIME} {owner}.xlsx', engine='xlsxwriter') as writer:
            OA_data.to_excel(writer, index=False, sheet_name='AR_ALL', engine='xlsxwriter')
    else:
        print(f'OA_data: {len(OA_data)} OA_SF: {len(OA_SF)} ')
        
        with pd.ExcelWriter(f'./Output/AR_OA/AR {Acc_date} Time {AR_TIME} {owner}.xlsx', engine='xlsxwriter') as writer:
            print("startsaveOA_data")
            OA_data.to_excel(writer, index=False, sheet_name='AR_ALL', engine='xlsxwriter')
            print("startsaveOA_SF")
            OA_SF.to_excel(writer, index=False, sheet_name='AR_SF', engine='xlsxwriter') 
    path = f'./Output/AR_OA/AR {Acc_date} Time {AR_TIME} {owner}.xlsx'
    set_excel_file_password(path, password)
logger.info('Exported AR and AR_SF done!!')

========owner======== GMV
========owner======== ACT
========owner======== KOR
OA_data: 3657 OA_SF: 3169 
startsaveOA_data
startsaveOA_SF
========owner======== KWC
========owner======== MGC
OA_data: 4940 OA_SF: 3191 


PermissionError: [Errno 13] Permission denied: './Output/AR_OA/AR 20240826 Time 0830 MGC.xlsx'

In [48]:
AR_OA = AR[~AR['OA'].isna()]
OA = AR_OA['OA'].unique()
password = input('กรุณาระบุ password: ')

for owner in OA:
        
    print('========owner========')
    OA_data = AR_OA[AR_OA['OA'] == owner]
    print('========finishOA_datafitering========')
    OA_SF = OA_data[OA_data['Product Code'].str.contains('SF', na=False)]
    print('========finishOA_SF========')
    OA_data = OA_data[~OA_data['Product Code'].str.contains('SF', na=False)]
    print('========finishOA_data========')
    
    
    if OA_SF.shape[0] == 0:
        with pd.ExcelWriter(f'./Output/AR_OA/AR {Acc_date} Time {AR_TIME} {owner}.xlsx', engine='xlsxwriter') as writer:
            OA_data.to_excel(writer, index=False, sheet_name='AR_ALL', engine='xlsxwriter')
    else:
        with pd.ExcelWriter(f'./Output/AR_OA/AR {Acc_date} Time {AR_TIME} {owner}.xlsx', engine='xlsxwriter') as writer:
            OA_data.to_excel(writer, index=False, sheet_name='AR_ALL', engine='xlsxwriter')
            OA_SF.to_excel(writer, index=False, sheet_name='AR_SF', engine='xlsxwriter') 
    path = f'./Output/AR_OA/AR {Acc_date} Time {AR_TIME} {owner}.xlsx'
    set_excel_file_password(path, password)
logger.info('Exported AR and AR_SF done!!')

========owner========
========owner========
========owner========
========owner========
========owner========


KeyboardInterrupt: 

In [ ]:
def export_file(AR, AR_TIME):
    AR_SF = AR[AR['Product Code'].str.contains('SF', na=False)].reset_index(drop=True)
    AR_ALL = AR[~AR['Product Code'].str.contains('SF', na=False)]
    AR_SF1 = AR_SF[:1000001]
    AR_SF2 = AR_SF[1000001:]
    print('========start========')
    AR_SF_ASSIGN = AR_SF[~AR_SF['OA'].isna()]
    logger.info('Exportting AR and AR_SF..')
    
    with pd.ExcelWriter(f'./Output/AR_TOTAL/AR {Acc_date} Time {AR_TIME}.xlsx', engine='xlsxwriter') as writer:
        step_start = time.time()
        AR_ALL.to_excel(writer, index=False, sheet_name='AR_ALL')
        print(f'Finished AR_ALL in {time.time() - step_start:.2f} seconds')
        
        step_start = time.time()
        AR_SF1.to_excel(writer, index=False, sheet_name='AR_SF1')
        print(f'Finished AR_SF1 in {time.time() - step_start:.2f} seconds')

        step_start = time.time()
        AR_SF2.to_excel(writer, index=False, sheet_name='AR_SF1')  # Fixed duplicate sheet name
        print(f'Finished AR_SF2 in {time.time() - step_start:.2f} seconds')

        step_start = time.time()
        AR_SF_ASSIGN.to_excel(writer, index=False, sheet_name='AR_SF_ASSIGN')
        print(f'Finished AR_SF_ASSIGN in {time.time() - step_start:.2f} seconds')

    end_time = time.time()
            
        
    print('========SAVETOTAL========')
    AR_OA = AR[~AR['OA'].isna()]
    OA = AR_OA['OA'].unique()
    password = input('กรุณาระบุ password: ')
    
    
    print('========STARTLOOP========')
    for owner in OA:
        
        print('========owner========')
        OA_data = AR_OA[AR_OA['OA'] == owner]
        OA_SF = OA_data[OA_data['Product Code'].str.contains('SF', na=False)]
        OA_data = OA_data[~OA_data['Product Code'].str.contains('SF', na=False)]
        if OA_SF.shape[0] == 0:
            with pd.ExcelWriter(f'./Output/AR_OA/AR {Acc_date} Time {AR_TIME} {owner}.xlsx', engine='xlsxwriter') as writer:
                OA_data.to_excel(writer, index=False, sheet_name='AR_ALL', engine='xlsxwriter')
        else:
            with pd.ExcelWriter(f'./Output/AR_OA/AR {Acc_date} Time {AR_TIME} {owner}.xlsx', engine='xlsxwriter') as writer:
                OA_data.to_excel(writer, index=False, sheet_name='AR_ALL', engine='xlsxwriter')
                OA_SF.to_excel(writer, index=False, sheet_name='AR_SF', engine='xlsxwriter') 
        path = f'./Output/AR_OA/AR {Acc_date} Time {AR_TIME} {owner}.xlsx'
        set_excel_file_password(path, password)
    logger.info('Exported AR and AR_SF done!!')

In [39]:
AR_SF = AR[AR['Product Code'].str.contains('SF', na=False)].reset_index(drop=True)
AR_ALL = AR[~AR['Product Code'].str.contains('SF', na=False)]
AR_SF1 = AR_SF[:1000001]
AR_SF2 = AR_SF[1000001:]
print('========start========')
AR_SF_ASSIGN = AR_SF[~AR_SF['OA'].isna()]
print('========finishassign========')
logger.info('Exportting AR and AR_SF..')



2025-03-04 17:29:18,278 - INFO - Exportting AR and AR_SF..


========start========
========finishassign========


In [ ]:
with pd.ExcelWriter(f'./Output/AR_TOTAL/AR {Acc_date} Time {AR_TIME}.xlsx', engine='xlsxwriter') as writer:
    print('========startALL========')
    AR_ALL.to_excel(writer, index=False, sheet_name='AR_ALL', engine='xlsxwriter')
    print('========finishALL========')
    
    
    AR_SF1.to_excel(writer, index=False, sheet_name='AR_SF1', engine='xlsxwriter')
    print('========finishSF1========') 
    AR_SF2.to_excel(writer, index=False, sheet_name='AR_SF1', engine='xlsxwriter')
    print('========finishSF2========') 
    AR_SF_ASSIGN.to_excel(writer, index=False, sheet_name='AR_SF_ASSIGN', engine='xlsxwriter')
    print('========AR_SF_ASSIGN========') 

========startALL========


Exception ignored in: <function ZipFile.__del__ at 0x000001F69F6B8EA0>
Traceback (most recent call last):
  File "c:\Users\Sireetron\AppData\Local\Programs\Python\Python313\Lib\zipfile\__init__.py", line 1980, in __del__
  File "c:\Users\Sireetron\AppData\Local\Programs\Python\Python313\Lib\zipfile\__init__.py", line 1997, in close
ValueError: seek of closed file


========finishALL========


In [33]:
export_file(AR, AR_TIME)

2025-03-04 16:36:08,358 - INFO - Exportting AR and AR_SF..


========start========
========SAVETOTAL========
========STARTLOOP========
========owner========
========owner========
========owner========
========owner========
========owner========
========owner========
========owner========
========owner========
========owner========
========owner========
========owner========
========owner========
========owner========
========owner========
========owner========
========owner========


2025-03-04 17:14:02,544 - INFO - Exported AR and AR_SF done!!
